In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

In [2]:
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#config = tf.compat.v1.ConfigProto(device_count = {'GPU': 1})

## Preliminary work on the dataset

In [3]:
yahoo_df = pd.read_csv("../data/OIL.csv")
yahoo_df = yahoo_df.set_index("Date")
yahoo_df = yahoo_df.astype(float,errors="raise")
yahoo_df

,Open,Close,vwap,SO,ROC_1,ROC_2,ROC_3,ROC_4,ROC_5,ROC_6,...,SRSI_10,SRSI_14,SRSI_30,Williams_14,ATR_14,CCI,APO,Differenced,Previous_differenced,principalDf
Date,,,,,,,,,,,,,,,,,,,,,
2001-05-14,28.700001,28.709999,28.102906,88.942238,0.560420,0.666194,1.700317,4.819276,3.384943,1.234127,...,0.574655,0.714322,0.859743,-12.500032,0.803378,71.355898,0.247115,0.005604,0.001052,-2.374259
2001-05-15,28.700001,28.980000,28.257060,100.000000,0.940440,1.506131,1.612900,2.656748,5.805039,4.357217,...,0.545729,0.701135,0.991615,-4.455452,0.757594,111.885803,0.232436,0.009404,0.005604,-3.404757
2001-05-16,29.180000,28.860001,28.369739,92.452898,-0.414075,0.522471,1.085819,1.192146,2.231672,5.366927,...,0.740228,0.564786,0.921306,-17.727239,0.763248,95.412027,0.207308,-0.004141,0.009404,-2.476551
2001-05-17,28.750000,28.910000,28.391519,95.597504,0.173248,-0.241545,0.696624,1.260948,1.367459,2.408786,...,0.645334,0.596789,0.833320,-15.454547,0.714815,85.150646,0.188269,0.001732,-0.004141,-2.272075
2001-05-18,29.150000,29.910000,28.479156,100.000000,3.459011,3.638251,3.209111,4.179731,4.763575,4.873771,...,0.808485,0.682910,0.919437,-2.721085,0.763506,150.138416,0.297564,0.034590,0.001732,-5.190917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-21,105.129997,112.120003,109.045494,59.595269,7.086921,8.875509,17.971382,16.258814,8.843802,2.551908,...,0.435757,0.546563,0.595820,-49.715977,8.523589,37.328302,6.536026,0.070869,0.016702,-6.458255
2022-03-22,112.900002,111.760002,109.843552,58.339161,-0.321085,6.743081,8.525926,17.592594,15.885524,8.494321,...,0.386760,0.494216,0.600186,-50.689741,8.148443,49.386949,5.491667,-0.003211,0.070869,-6.212631
2022-03-23,108.849998,114.930000,110.016440,69.399868,2.836434,2.506241,9.770777,11.604192,20.928029,19.172540,...,0.576180,0.541041,0.628693,-42.115226,7.997985,50.618855,4.370321,0.028364,-0.003211,-8.544433


# Train, test, val

In [4]:

X = yahoo_df[['ROC_1', 'ROC_4', 'ROC_5', 'ROC_30', 'Return', '2 Day ROI', '3 Day ROI',
       '4 Day ROI', '30 Day ROI', 'RSI_1', 'RSI_2', 'RSI_3', 'RSI_4',
       'MACDs_12_26_9', 'SRSI_14', 'SRSI_30', 'CCI', 'Previous_differenced',
       'principalDf']]
y = yahoo_df["Up down"]

In [5]:
# Split in 80/20 the dataframeX
X_train_80, X_test, y_train_80, y_test = train_test_split(X, y, test_size = 0.20,
                                                          shuffle=False)

# Split in 75/25 the remaining 80 %
X_train, X_valid, y_train, y_valid = train_test_split(X_train_80, y_train_80,
                                                      test_size = 0.25, shuffle=False)

# Normalize

In [6]:
def normalize(df):
    #df = data
    for column in df:
        df[column]=((df[column]-df[column].mean())/df[column].std())
    return df

In [7]:
X_train_80 = normalize(X_train_80)
X_test = normalize(X_test)


In [8]:
X_train = normalize(X_train)
X_valid = normalize(X_valid)

# Light Gradient Boosting Machine


In [9]:
# Build LightGBM Model
valid_data = lgb.Dataset(X_valid, label = y_valid)
train_data = lgb.Dataset(X_train, label = y_train)
test_data = lgb.Dataset(X_train_80, label = y_train_80)

In [10]:
# Select Hyper-Parameters
params = {'boosting_type': 'dart',
          "num_leaves":[2],
          'max_depth' : [10],
          'objective': 'multiclass',
          'nthread': 4,
          'learning_rate': 0.0125,
          'max_bin': 100,
          'reg_lambda': 1.4,
          'num_class' : 2,  
          'metric' : 'multi_logloss'}

In [11]:
# Train the model
lgbm = lgb.train(params, train_data)

# Predict
pred_train = lgbm.predict(X_train)
pred_valid = lgbm.predict(X_valid)

y_pred_train = np.argmax(pred_train, axis = 1)
y_pred_valid = np.argmax(pred_valid, axis = 1)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000446 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1900
[LightGBM] [Info] Number of data points in the train set: 3144, number of used features: 19
[LightGBM] [Info] Start training from score -0.732724
[LightGBM] [Info] Start training from score -0.655077
Train Accuracy: 0.53562 - Validation Accuracy: 0.51145


# Tuning very slow

In [12]:

param_test ={
            'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 100

#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='roc_auc',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

gs.fit(X_train, y_train)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

In [13]:
params = {'boosting_type': 'dart',
          "learning_rate":0.001,
          "num_iterations":1000,
        'colsample_bytree': 0.9242593373375023,
          'min_child_samples': 282,
          'min_child_weight': 10.0, 
          'num_leaves': 36, 
          'reg_alpha': 0, 
          'reg_lambda': 0.1, 
          'subsample': 0.4955174534902478,
          'objective': 'multiclass',
          'metric' : 'multi_logloss',
          'num_class' : 2, #Away, Draw and Home,
         }

In [14]:
len(pred_train)

3144

In [15]:
# Train the model
lgbm = lgb.train(params, train_data)

# Predict
pred_train = lgbm.predict(X_train)
pred_valid = lgbm.predict(X_valid)

y_pred_train = np.argmax(pred_train, axis = 1)
y_pred_valid = np.argmax(pred_valid, axis = 1)

# Compute accuracy
train_acc = accuracy_score(y_train, y_pred_train)
valid_acc = accuracy_score(y_valid, y_pred_valid)
print("Train Accuracy: {:.5f} - Validation Accuracy: {:.5f}".format(train_acc, valid_acc))

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005456 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1900
[LightGBM] [Info] Number of data points in the train set: 3144, number of used features: 19
[LightGBM] [Info] Start training from score -0.732724
[LightGBM] [Info] Start training from score -0.655077
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

C:\Users\enric\anaconda3\envs\data_science\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Train Accuracy: 0.56711 -

In [16]:
# Predict
pred_test = lgbm.predict(X_test)

y_pred_test = np.argmax(pred_test, axis = 1)

# Compute accuracy
test_acc = accuracy_score(y_test, y_pred_test)
print("Test Accuracy: {:.5f}".format(test_acc))

Test Accuracy: 0.53384


# Save accuracies

In [17]:
# Train the test set

# Predict
pred = lgbm.predict(X_test)
y_pred = np.argmax(pred, axis = 1)

# Compute accuracy
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.5f}".format(acc))

# Store the test and the validation accuracy
LGBM = acc

Test Accuracy: 0.53384


In [18]:
df_prediction = pd.read_csv("../data/model_accuracy_oil.csv", index_col = False)
df_prediction =  df_prediction.drop(columns=["Unnamed: 0"])

In [19]:
#df_prediction = pd.read_csv("../data/model_accuracy.csv", index_col = False)
df_prediction["LGBM_pred"] = y_pred
df_prediction["LGBM_accuracy"] = [LGBM for x in range(len(y_pred))]
df_prediction["LGBM_prob"] = pred[:,1]
df_prediction

,ANN_pred,ANN_accuracy,ANN_prob,y,kNN_pred,kNN_accuracy,kNN_prob,RF_pred,RF_accuracy,RF_prob,XGB_pred,XGB_accuracy,XGB_prob,LGBM_pred,LGBM_accuracy,LGBM_prob
0,0,0.469018,0.404047,0,1,0.488084,1.0,0,0.514776,0.440374,0,0.521449,0.499819,1,0.533842,0.513085
1,0,0.469018,0.342436,1,0,0.488084,0.0,1,0.514776,0.558104,1,0.521449,0.511250,1,0.533842,0.513026
2,0,0.469018,0.339922,1,0,0.488084,0.0,1,0.514776,0.565385,0,0.521449,0.494657,1,0.533842,0.513026
3,0,0.469018,0.404326,0,1,0.488084,1.0,1,0.514776,0.522251,1,0.521449,0.500281,1,0.533842,0.504916
4,0,0.469018,0.361674,0,1,0.488084,1.0,1,0.514776,0.583792,1,0.521449,0.529373,1,0.533842,0.515756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,0,0.469018,0.448136,0,0,0.488084,0.5,0,0.514776,0.470678,1,0.521449,0.549180,1,0.533842,0.506586
1045,0,0.469018,0.448477,1,0,0.488084,0.5,0,0.514776,0.485184,1,0.521449,0.531798,1,0.533842,0.505718
1046,0,0.469018,0.438599,0,0,0.488084,0.5,0,0.514776,0.466790,1,0.521449,0.512161,1,0.533842,0.506459
1047,0,0.469018,0.417918,1,0,0.488084,0.0,1,0.514776,0.589712,1,0.521449,0.542420,1,0.533842,0.508596


In [20]:
df_prediction.to_csv("../data/model_accuracy_oil.csv")
